In [1]:
from keras.datasets import mnist

(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

Using TensorFlow backend.


11501568/11490434 [==============================] - 0s 0us/step


In [0]:
import numpy as np
# train and test images in shape (60000, 28, 28)
train_images_28x28 = train_images_original.astype('float32') / 255
test_images_28x28 = test_images_original.astype('float32') / 255

train_images_rounded = np.round(train_images_28x28)
test_images_rounded = np.round(test_images_28x28)

# train and test images in shape (60000, 784)
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

In [0]:
def neighbours(x, y, zeros):
    DIRC = [(0, 1), (1, 0), (0, -1), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)]
    ROW = 28
    COL = 28

    nb = []
    for d in DIRC:
        dx, dy = d
        new_x = x + dx
        new_y = y + dy
        if 0 <= new_x < COL and 0 <= new_y < ROW:
            if [new_x, new_y] in zeros:
                new_point = [new_x, new_y]
                nb.append(new_point)
    return nb

In [0]:
def islands(zeros):
    visited = []
    count = 0

    for p in zeros:
        p = [p[0], p[1]]
        if p not in visited:
            count += 1
            dfs(p, visited, zeros)

    return float(count)

In [0]:
def dfs(point, visited, zeros):
    point = [point[0], point[1]]
    if point in visited:
        return
    else:
        visited.append(point)
        nb = neighbours(point[0], point[1], zeros)

        for n in nb:
            dfs(n, visited, zeros)

In [6]:
train_images_featured = np.empty((0, 785))
for i in range(train_images.shape[0]):
    if i % 10000 == 0:
        print(i)
    allZeros = np.argwhere(train_images_rounded[i] == 0.0)
    allZeros = allZeros.tolist()
    islandsNum = islands(allZeros)
    im = np.append(train_images[i], islandsNum)
    train_images_featured = np.vstack([train_images_featured, im])
    

0
10000
20000
30000
40000
50000


In [0]:
test_images_featured = np.empty((0, 785))
for i in range(test_images.shape[0]):
    if i % 1000 == 0:
        print(i)
    allZeros = np.argwhere(test_images_rounded[i] == 0.0)
    allZeros = allZeros.tolist()
    islandsNum = islands(allZeros)
    im = np.append(test_images[i], islandsNum)
    test_images_featured = np.vstack([test_images_featured, im])


In [8]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(785,)))
network.add(layers.Dense(10, activation='softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
network.compile(optimizer='sgd',
                loss='categorical_crossentropy',
                metrics=['accuracy'])


In [0]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)


In [11]:
epochs = 10
history = network.fit(train_images_featured, 
                      train_labels, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(train_images_featured, train_labels))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 23s 376us/step - loss: 1.0858 - acc: 0.7549 - val_loss: 0.5991 - val_acc: 0.8679
Epoch 2/10
60000/60000 [==============================] - 22s 367us/step - loss: 0.4943 - acc: 0.8841 - val_loss: 0.4241 - val_acc: 0.8960
Epoch 3/10
60000/60000 [==============================] - 20s 341us/step - loss: 0.3886 - acc: 0.9015 - val_loss: 0.3594 - val_acc: 0.9068
Epoch 4/10
60000/60000 [==============================] - 21s 346us/step - loss: 0.3408 - acc: 0.9106 - val_loss: 0.3235 - val_acc: 0.9148
Epoch 5/10
60000/60000 [==============================] - 21s 346us/step - loss: 0.3117 - acc: 0.9165 - val_loss: 0.2998 - val_acc: 0.9195
Epoch 6/10
60000/60000 [==============================] - 21s 345us/step - loss: 0.2913 - acc: 0.9216 - val_loss: 0.2823 - val_acc: 0.9238
Epoch 7/10
60000/60000 [==============================] - 21s 348us/

In [12]:
loss, accuracy = network.evaluate(train_images_featured, train_labels, verbose=False)
print("Test loss: ", round(loss, 3))
print("Test accuracy: ", round(accuracy, 3))


('Test loss: ', 0.239)
('Test accuracy: ', 0.934)
